# Truss Simulation

A Simple Truss Structure Finite Element Analysis Programme  
  
Created by: A Dickinson, 18/02/2022  
Edited by: Lee Kang Jie 01/03/2023  

# Import modules

In [ ]:
import numpy as np
import matplotlib.pyplot as pl
import matplotlib.axes as ax
import pandas as pd
import matplotlib.colors as colors
import matplotlib.cm as cmx
import matplotlib as mpl

# replace inline with qt to have plots appearing in a separate window
%matplotlib inline
np.set_printoptions(precision=1)
#np.set_printoptions(suppress=True)

# Inputs

In [ ]:
# Young's Modulus
E = 207e9

# Cross-sectional area of rod
AR1 = np.pi * 20e-3**2
AR2 = np.pi * 25e-3**2

A = [AR1, AR1, AR2, AR2, AR2, AR2, AR2, AR1, AR2, AR2, AR2]
E = [E, E, E, E, E, E, E, E, E, E, E]

# Coordinates
X = [0, 1, 2, 0.5, 1.5, 3, 2.5]
Y = [0, 0, 0, 0.8, 1, 0, 0.8]

nodes = np.array(list(zip(X,Y)))

elemcon = np.array([[1, 2],
                    [2, 3],
                    [1, 4],
                    [2, 4],
                    [2, 5],
                    [3, 5],
                    [4, 5],
                    [3, 6], #
                    [3, 7], #
                    [5, 7], #
                    [6, 7]])#

#Define Boundary Conditions. Set Nodes with zero DoFs:
XBCs = np.array([1, 6])
YBCs = np.array([1, 6])

#Define Loads.
Loads = np.array([[0.0, 0.0],
                  [0.0, -10000],
                  [0.0, -10000],
                  [0.0, 0.0],
                  [0.0, 0.0],
                  [0.0, 0.0], #
                  [0.0, 0.0]])#

# Solution class

In [ ]:
class Truss:
    
    
    def __init__(self, nodes, elemcon, A, E, Loads, XBCs, YBCs):
        self.nodes = nodes
        self.elemcon = elemcon
        self.A = A
        self.E = E
        self.elems = self.create_elems()
        self.Loads = Loads 
        self.XBCs = XBCs
        self.YBCs = YBCs
        self.K_global = self.calculation()[0]
        self.Displacements = self.calculation()[1]
        self.Selem = self.calculation()[2]
         
    
    def create_elems(self):
        elems = pd.DataFrame({'A': self.A, 'E': self.E})
        elems['Node 1'] = [self.nodes[i[0]-1] for i in self.elemcon]
        elems['Node 2'] = [self.nodes[i[1]-1] for i in self.elemcon]
        elems['L'] = [np.linalg.norm(node1 - node2) for node1, node2 in zip(elems['Node 1'], elems['Node 2'])]
        elems['k'] = [elem_E * elem_A / elem_L for elem_E, elem_A, elem_L in zip(elems['E'], elems['A'], elems['L'])]
        elems['theta'] = [np.arctan2((nodes[i[1]-1][1] - nodes[i[0]-1][1]), (nodes[i[1]-1][0] - nodes[i[0]-1][0])) for i in elemcon]
        return elems
    
    
    def calculation(self):
        n_nodes = len(self.nodes)
        K_global = np.zeros((n_nodes * 2, n_nodes * 2))

        # Calculate Stresses:
        Felem = np.zeros(len(self.elemcon))
        Selem = np.zeros(len(self.elemcon))

        for i, row in self.elems.iterrows():
            node1, node2 = row['Node 1'], row['Node 2']
            c, s = np.cos(row['theta']), np.sin(row['theta'])
            # Transformation matrix
            T = np.array([[c, -s, 0, 0],
                          [s, c, 0, 0],
                          [0, 0, c, -s],
                          [0, 0, s, c]])

            k = row['k']
            KLocal = np.array([[k,0,-k,0],
                               [0,0,0,0],
                               [-k,0,k,0],
                               [0,0,0,0]])
            n1, n2 = self.elemcon[i, 0] - 1, self.elemcon[i, 1] - 1
            DOF = np.array([[0, 2*n1],
                            [1, 2*n1 + 1],
                            [2, 2*n2],
                            [3, 2*n2 + 1]])
            KElGlobal = T @ KLocal @ np.linalg.inv(T)

            # Assemble Element Stiffness Matrices:
            K_global[DOF[:, 1][:, None], DOF[:, 1]] += KElGlobal[DOF[:, 0][:, None], DOF[:, 0]]

        # Create BCs cutting matrix:
        BCs = np.identity(len(self.nodes) * 2)
        FixedDoFs = np.zeros(len(self.nodes) * 2)  # Create a binary vector with 1s where DoFs are fixed
        FixedDoFs[2*(self.XBCs[:]-1)]=1 #enter fixed x DoFs
        FixedDoFs[2*(self.YBCs[:]-1)+1]=1 #enter fixed y DoFs
        F_global=np.reshape(self.Loads,[len(self.nodes)*2,1])


        for x in range(len(self.nodes)*2-1, -1, -1):
            if FixedDoFs[x]==1:          #if BC is fixed
                BCs=np.delete(BCs, x, 1) #remove column

        #Reduce the Stiffness Matrix by removing fixed DoFs:
        K_reduced=np.matmul(np.transpose(BCs),K_global) #Removes Rows
        K_reduced=np.matmul(K_reduced,BCs) #Removes Columns

        #Reduce the Loads List
        F_reduced=np.matmul(np.transpose(BCs),F_global) #Removes Rows

        DisplacementsReduced=np.matmul(np.linalg.inv(K_reduced),F_reduced)
        #print(DisplacementsReduced)

        #Reintroduce the zero displacements at fixed DoFs:
        Displacements=np.matmul(BCs,DisplacementsReduced)

 
        for i, row in self.elems.iterrows():
            node1, node2 = row['Node 1'], row['Node 2']
            c, s = np.cos(row['theta']), np.sin(row['theta'])
            # Transformation matrix
            T = np.array([[c, -s, 0, 0],
                          [s, c, 0, 0],
                          [0, 0, c, -s],
                          [0, 0, s, c]])

            k = row['k']
            KLocal = np.array([[k,0,-k,0],
                               [0,0,0,0],
                               [-k,0,k,0],
                               [0,0,0,0]])
            n1, n2 = self.elemcon[i, 0] - 1, self.elemcon[i, 1] - 1
            DOF = np.array([[0, 2*n1],
                            [1, 2*n1 + 1],
                            [2, 2*n2],
                            [3, 2*n2 + 1]])
            # Get global displacement vector for the element:
            Uelem = np.zeros([4, 1])
            Uelem[:] = Displacements[DOF[:, 1]]

            # then local displacement vector for the element
            uelem = np.zeros([4, 1])
            uelem = np.matmul(np.linalg.inv(T), Uelem)

            # then elemental forces, and finally
            felem = np.zeros([4, 1])
            felem[:] = np.matmul(KLocal, uelem)
            Felem[i] = felem[2]

            # elemental stresses
            Selem[i] = felem[2] / self.A[i]

        Selem[:] /= 1000000  # Convert to MPa

        return K_global, Displacements, Selem
     
        
    def plottings(self,nodenums=1,elemnums=1,loads=0,deformations=0,dscale=0):       
        """
        Plot the finite element model geometry using nodal coordinates in a 2 x n array,
        and element connectivity in a 2 x m array of integers. Additional arguments are
        binary settings to control display of node or element numbers, defaulting on (=1).
        """
        #Plot the Mesh
        pl.figure()
        #Plot nodes
        pl.plot(self.nodes[:, 0], self.nodes[:, 1], 'ro', zorder=3)
        #Plot elements one by one. Note, node numbers have '-1' as numbered from 1, vs. python numbered from 0.
        pl.plot([self.nodes[self.elemcon[:,0]-1,0],self.nodes[self.elemcon[:,1]-1,0]],[self.nodes[self.elemcon[:,0]-1,1],self.nodes[self.elemcon[:,1]-1,1]],'k', zorder=2)
        pl.gca().set_aspect(1)
        pl.grid(1)

        if nodenums:
            #Plot node numbers
            for x in range(len(self.nodes)):
                nodename='n'+str(x+1)
                pl.text((self.nodes[x,0]),(self.nodes[x,1]-0.1),nodename,color='b')


        if elemnums:
            #Plot elem numbers
            for x in range(len(self.elemcon)):
                elemname='e'+str(x+1)
                pl.text((((self.nodes[self.elemcon[x,0]-1,0]+self.nodes[self.elemcon[x,1]-1,0])/2)),(((self.nodes[self.elemcon[x,0]-1,1]+self.nodes[self.elemcon[x,1]-1,1])/2)-0.1),elemname,color='k')

        if loads:
            #Plot load symbols:
            for x in range(len(self.Loads)):
                if np.any(self.Loads[x,:]) != 0:
                    pl.arrow(self.nodes[x, 0], self.nodes[x, 1], np.sign(self.Loads[x,0])*0.25, np.sign(self.Loads[x,1])*0.25, width = 0.05,head_width=0.1)
                    bottom, top = pl.ylim()  # return the current ylim                
                    left, right = pl.xlim()  # return the current xlim                
                    pl.ylim([bottom-0.25, top+0.25])
                    pl.xlim([left-0.25, right+0.25])

        if deformations:
            #Displacement may be too small to show, so we can
            #scale maximum displacements to dscale*longest element
            #for display, where dscale is a scale factor (exaggerate the displacement)
            MaxDisp=np.max(self.elems['L'])
            DisplacementsPlot=self.Displacements/np.amax(abs(self.Displacements))*MaxDisp*dscale
            nodesdisp = self.nodes + np.reshape(DisplacementsPlot,[len(self.nodes),2])

            #Plot nodes
            pl.plot(nodesdisp[:, 0], nodesdisp[:, 1], 'ro', zorder=3)
            #Plot elements one by one. Note, node numbers have '-1' as numbered from 1, vs. python numbered from 0.
            pl.plot([nodesdisp[self.elemcon[:,0]-1,0],nodesdisp[self.elemcon[:,1]-1,0]],[nodesdisp[self.elemcon[:,0]-1,1],nodesdisp[self.elemcon[:,1]-1,1]],'k--', zorder=2)
            
            
            #Define stress colour contours
            cmap=pl.cm.PiYG#PRGn
            ##Scale from min to max,
            #cNorm=colors.Normalize(vmin=np.min(Selem[:]), vmax=np.max(Selem[:]))
            #scalarMap=cmx.ScalarMappable(norm=cNorm,cmap=cmap)
            #or Scale from -max to max or min to -min
            max=np.max([np.abs(np.max(self.Selem[:])), np.abs(np.min(self.Selem[:]))])
            cNorm=colors.Normalize(-max, max)
            scalarMap=cmx.ScalarMappable(norm=cNorm,cmap=cmap)

            #Plot the Mesh
            fig=pl.figure()
            ax=fig.add_axes([0.1, 0.1, 0.9, 0.55]) # [left, bottom, width, height]
            axc=fig.add_axes([1.05, 0.15, 0.05, 0.45])

            #Plot nodes
            ax.plot(nodesdisp[:, 0], nodesdisp[:, 1], 'ko', zorder=3)
            #Plot elements one by one. Note, node numbers have '-1' as numbered from 1, vs. python numbered from 0.
            ax.plot([self.nodes[self.elemcon[:,0]-1,0],nodes[self.elemcon[:,1]-1,0]],[self.nodes[self.elemcon[:,0]-1,1],self.nodes[self.elemcon[:,1]-1,1]],'k:', zorder=2)

            #Plot elements one by one. Note, node numbers have '-1' as numbered from 1, vs. python numbered from 0.
            for x in range(len(self.elemcon)):
                colorval=scalarMap.to_rgba(self.Selem[x])
                ax.plot([nodesdisp[self.elemcon[x,0]-1,0],nodesdisp[self.elemcon[x,1]-1,0]],[nodesdisp[self.elemcon[x,0]-1,1],nodesdisp[self.elemcon[x,1]-1,1]],color=colorval, zorder=2)
                ax.set_aspect(1)
                ax.grid(1)

                elemstress=str(round(self.Selem[x], 2))+' MPa'
                ax.text((((nodesdisp[self.elemcon[x,0]-1,0]+nodesdisp[self.elemcon[x,1]-1,0])/2)),(((nodesdisp[self.elemcon[x,0]-1,1]+nodesdisp[self.elemcon[x,1]-1,1])/2)+0.05),elemstress,color='k')

            colorbar=mpl.colorbar.ColorbarBase(axc,cmap=cmap,norm=cNorm,orientation='vertical',label="$Stress\ (MPa)$")
        pl.show()

        return None

In [ ]:
truss = Truss(nodes, elemcon, A, E, Loads, XBCs, YBCs)
truss.plottings(0,0,0,1,0.15)